# Crypto Data Analysis

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential


In [3]:
df = pd.read_csv('365daysData.csv')

In [4]:
df.head(5)

,Time,Open,High,Low,Close,Volume
0,2021-05-20 14:44:00,41872.00,41928.90,41820.61,41828.55,111.365942
1,2021-05-20 14:45:00,41828.32,41973.15,41824.95,41898.79,93.760845
2,2021-05-20 14:46:00,41897.14,41938.13,41801.55,41870.24,107.078372
3,2021-05-20 14:47:00,41870.63,41870.63,41680.00,41738.49,134.848603
4,2021-05-20 14:48:00,41731.57,41793.36,41660.12,41783.22,75.213453


In [5]:
df

,Time,Open,High,Low,Close,Volume
0,2021-05-20 14:44:00,41872.00,41928.90,41820.61,41828.55,111.365942
1,2021-05-20 14:45:00,41828.32,41973.15,41824.95,41898.79,93.760845
2,2021-05-20 14:46:00,41897.14,41938.13,41801.55,41870.24,107.078372
3,2021-05-20 14:47:00,41870.63,41870.63,41680.00,41738.49,134.848603
4,2021-05-20 14:48:00,41731.57,41793.36,41660.12,41783.22,75.213453
...,...,...,...,...,...,...
525214,2022-05-20 14:48:00,29726.26,29762.15,29713.63,29762.15,76.603140
525215,2022-05-20 14:49:00,29762.15,29763.99,29601.00,29606.73,338.318660
525216,2022-05-20 14:50:00,29606.73,29672.90,29595.80,29595.88,141.983710
525217,2022-05-20 14:51:00,29595.89,29625.95,29530.00,29530.01,154.166900


In [6]:
df.shape

(525219, 6)

In [7]:
df['Volume'].unique

<bound method Series.unique of 0         111.365942
1          93.760845
2         107.078372
3         134.848603
4          75.213453
             ...    
525214     76.603140
525215    338.318660
525216    141.983710
525217    154.166900
525218    428.338970
Name: Volume, Length: 525219, dtype: float64>

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525219 entries, 0 to 525218
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    525219 non-null  object 
 1   Open    525219 non-null  float64
 2   High    525219 non-null  float64
 3   Low     525219 non-null  float64
 4   Close   525219 non-null  float64
 5   Volume  525219 non-null  float64
dtypes: float64(5), object(1)
memory usage: 24.0+ MB


In [9]:
df.describe()

,Open,High,Low,Close,Volume
count,525219.000000,525219.000000,525219.000000,525219.000000,525219.000000
mean,44105.885289,44132.825623,44079.126725,44105.858220,38.918824
std,8729.522087,8731.015451,8727.420888,8729.546929,60.345469
min,26797.320000,26954.210000,26700.000000,26771.950000,0.000000
25%,38116.340000,38146.285000,38088.040000,38116.240000,13.077140
50%,42575.860000,42596.180000,42555.600000,42575.870000,22.639440
75%,48239.480000,48266.950000,48212.640000,48239.485000,42.018110
max,69000.000000,69000.000000,68786.700000,69000.000000,2636.713888


In [10]:
df = df[501219:521219]
test_df = df[521219:]

In [11]:
df.shape

(20000, 6)

In [12]:
scaler = MinMaxScaler(feature_range=(0,1))
scaledData = scaler.fit_transform(df['Close'].values.reshape(-1,1))

In [13]:
scaledData

array([[0.83343865],
       [0.8340872 ],
       [0.83322423],
       ...,
       [0.24984771],
       [0.24990605],
       [0.25092283]])

In [14]:
prediction_days = 60
x_train,y_train = [],[]

for x in range(prediction_days, len(scaledData)):
    x_train.append(scaledData[x-prediction_days:x,0])
    y_train.append(scaledData[x,0])


In [15]:
x_train, y_train = np.array(x_train),np.array(y_train)

In [19]:
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_train.shape

(19940, 60, 1)

In [17]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train,y_train,epochs=25, batch_size=32)

Epoch 1/25
480/624 [======================>.......] - ETA: 6s - loss: 0.0049

KeyboardInterrupt: 